**SA463A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2021 &#x25aa; Uhan**

# Lesson 20. Strings and Datetimes in Pandas

## Overview

- Strings and datetimes can be awkward to work with, especially compared to numeric values


- In this lesson, we'll learn the basics of working with strings and datetimes in Pandas


- As we'll see, there are *many* methods related to string and datetime processing


- Consider this lesson just an introduction!

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## The UFO dataset 

* Let's start by importing Pandas:

In [1]:
import pandas as pd

- For this lesson, we'll use [this dataset on UFO sightings](http://bit.ly/uforeports), based on information from the [National UFO Reporting Center](http://www.nuforc.org/webreports.html)


- This dataset is also included in `data/ufo.csv`, in the same folder as this notebook


- Let's read the data into a DataFrame:

In [2]:
df = pd.read_csv('data/ufo.csv')

* Let's peek at the first 5 rows:

In [3]:
df.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


- Let's also get some more information about the data types (dtypes) in this DataFrame:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18241 entries, 0 to 18240
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   City             18216 non-null  object
 1   Colors Reported  2882 non-null   object
 2   Shape Reported   15597 non-null  object
 3   State            18241 non-null  object
 4   Time             18241 non-null  object
dtypes: object(5)
memory usage: 712.7+ KB


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Working with strings

- The Series object is equipped with a set of **string methods** that perform string processing operations on the entire Series at once


- These methods also conveniently exclude missing values automatically, unlike their counterparts in the Python standard library


- You can access these methods with the `.str` attribute


- For example, `.str.lower()` converts strings to lowercase, like this:

In [5]:
# Solution
df['City'].str.lower()

0                      ithaca
1                 willingboro
2                     holyoke
3                     abilene
4        new york worlds fair
                 ...         
18236              grant park
18237             spirit lake
18238             eagle river
18239             eagle river
18240                    ybor
Name: City, Length: 18241, dtype: object

- Note that you can use string methods in `.query()`, like this:

In [6]:
# Solution
df.query('City.str.lower() == "ithaca"')

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
4068,Ithaca,NaN,CIGAR,NY,6/1/1979 19:00
5631,Ithaca,NaN,OTHER,MI,6/1/1987 17:00
6961,Ithaca,NaN,OTHER,NY,1/10/1993 0:30
7573,Ithaca,RED GREEN,LIGHT,NY,10/15/1994 18:00
9088,Ithaca,NaN,NaN,NY,2/16/1996 21:45
16537,Ithaca,NaN,FLASH,MI,6/3/2000 22:35
17049,Ithaca,NaN,TEARDROP,NY,7/30/2000 20:20


- Note that this kind of query might help if your data contains values with inconsistent capitalization

- You may recall that we've already seen a few other examples of Series string methods, like `.str.split()` and `.str.cat()`


- Here are some string methods that you might find particularly useful:

| `.str` method | Description |
| :- | :- |
| `.cat()` | Concatenate strings |
| `.split()` | Split strings on delimiter |
| `.contains()` | Return boolean array if each string contains pattern/regex |
| `.replace()` | Replace occurrences of pattern/regex/string with some other string or the return value of a callable given the occurrence |
| `.pad()` | Add whitespace to left, right, or both sides of strings |
| `.slice()` | Slice each string in the Series |
| `.slice_replace()` | Replace slice in each string with passed value |
| `.count()` | Count occurrences of pattern |
| `.startswith()` | Test if the start of each string element matches a pattern |
| `.endswith()` | Test if the end of each string element matches a pattern |
| `.len()` | Compute string lengths |
| `.strip()` | Strip whitespaces from left and right sides |
| `.partition()` | Split the string at the first occurrence of a substring |
| `.lower()` | Convert strings to lowercase |
| `.upper()` | Convert strings to uppercase |
| `.title()` | Convert strings to title case |
| `.find()` | Find substring within string |


- Note that this list is incomplete, and doesn't tell you how these methods work... (e.g. what arguments do they take?)


- For more information, [here is the section on Series string methods](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-str) in the Pandas documentation


- Some of the documentation make references to a **regular expression** or **regex**
    - These are specially constructed sequences of characters that define a search
    - Regexes can be very useful... and somewhat complicated to use
    - There are *many* resources on the internet for learning about regexes
    - For example, [here is a nice tool](https://regexr.com/) to learn and test regular expressions

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Working with datetimes

- Note that our UFO dataset contains a column `Time` with the date and time (*datetime*) of the sighting:

In [7]:
df.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


- Above, when we used `df.info()`, we saw that the column `Time` consists of strings


- Keeping the date and time information as a string is inconvenient &mdash; for example:
    - We can't easily group by datetime components, like month or hour
    - We can't easily perform datetime arithmetic (e.g. add a day to each datetime)


- One way to resolve this would be to use string methods


- For example, we can use `.str.split()` to tease out the different components of the column `Time`, like this:

In [8]:
# Solution
(
    df
    .assign(
        MMDDYYYY=lambda x: x['Time'].str.split(expand=True)[0],
        HHMM=lambda x: x['Time'].str.split(expand=True)[1],
        month=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[0],
        day=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[1],
        year=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[2],
        hour=lambda x: x['HHMM'].str.split(pat=':', expand=True)[0],
        minute=lambda x: x['HHMM'].str.split(pat=':', expand=True)[1],
    )
)

,City,Colors Reported,Shape Reported,State,Time,MMDDYYYY,HHMM,month,day,year,hour,minute
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,6/1/1930,22:00,6,1,1930,22,00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,6/30/1930,20:00,6,30,1930,20,00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,2/15/1931,14:00,2,15,1931,14,00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,6/1/1931,13:00,6,1,1931,13,00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,4/18/1933,19:00,4,18,1933,19,00
...,...,...,...,...,...,...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,12/31/2000,23:00,12,31,2000,23,00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,12/31/2000,23:00,12,31,2000,23,00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,12/31/2000,23:45,12,31,2000,23,45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,12/31/2000,23:45,12,31,2000,23,45


- An arguably better alternative would be to use `pd.to_datetime()` to convert a column with well-formatted strings to the **datetime64** dtype


- Pandas has a variety of built-in tools to manipulate dates and times represented with the datetime64 dtype
    

- Back to our example: we can convert the column `Time` to datetime64 with `pd.to_datetime()` by specifying the format of the dates and times in `Time`:

In [9]:
# Solution
new_df = (
    df
    .assign(
        datetime=lambda x: pd.to_datetime(x['Time'], format='%m/%d/%Y %H:%M')
    )
)

new_df.head()

,City,Colors Reported,Shape Reported,State,Time,datetime
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,1933-04-18 19:00:00


- We can use `.info()` to confirm that the `datetime` column does indeed contain datetime64 values:

In [10]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18241 entries, 0 to 18240
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   City             18216 non-null  object        
 1   Colors Reported  2882 non-null   object        
 2   Shape Reported   15597 non-null  object        
 3   State            18241 non-null  object        
 4   Time             18241 non-null  object        
 5   datetime         18241 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 855.2+ KB


- The `format=...` keyword argument takes a string with standard C datetime format codes as placeholders for datetime components such as year, month day, etc.
    - [Here is a list of standard C datetime format codes](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) from the Python documentation

- The Series object has a variety of attributes and methods to work with datetime dtypes


- We access these attributes and methods with the `.dt` attribute


- For example, we can get the month of each date in `datetime` like this:

In [11]:
# Solution
new_df['datetime'].dt.month

0         6
1         6
2         2
3         6
4         4
         ..
18236    12
18237    12
18238    12
18239    12
18240    12
Name: datetime, Length: 18241, dtype: int64

- Here are some useful datetime attributes:

| `.dt` attribute | Description |
| :- | :- |
| `.year` | The year of the datetime |
| `.month` | The month as January=1, ..., December=12 |
| `.day` | The day of the datetime |
| `.hour` | The hours of the datetime |
| `.minute` | The minutes of the datetime |
| `.second` | The seconds of the datetime |
| `.microsecond` | The microseconds of the datetime |
| `.nanosecond` | The nanoseconds of the datetime |
| `.dayofweek` | The day of the week with Monday=0, Sunday=6 |


- For more information, [here is the section on datetime attributes and methods](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt) in the Pandas documentation 


- In addition, [here is the documentation for `pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)


- As you can see, there are many ways to convert datetime information into the datetime dtype


- For example, we could have taken the year-month-day-hour-minute information we parsed manually with string methods and passed those to `pd.to_datetime()` like this:

In [12]:
(
    df
    .assign(
        MMDDYYYY=lambda x: x['Time'].str.split(expand=True)[0],
        HHMM=lambda x: x['Time'].str.split(expand=True)[1],
        month=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[0],
        day=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[1],
        year=lambda x: x['MMDDYYYY'].str.split(pat='/', expand=True)[2],
        hour=lambda x: x['HHMM'].str.split(pat=':', expand=True)[0],
        minute=lambda x: x['HHMM'].str.split(pat=':', expand=True)[1],
        
        # We can pass a DataFrame with columns corresponding to 
        # year, month, day, hour, and minute into pd.to_datetime()
        datetime=lambda x: pd.to_datetime(x[['year', 'month', 'day', 'hour', 'minute']])
    )
)

,City,Colors Reported,Shape Reported,State,Time,MMDDYYYY,HHMM,month,day,year,hour,minute,datetime
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,6/1/1930,22:00,6,1,1930,22,00,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,6/30/1930,20:00,6,30,1930,20,00,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,2/15/1931,14:00,2,15,1931,14,00,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,6/1/1931,13:00,6,1,1931,13,00,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,4/18/1933,19:00,4,18,1933,19,00,1933-04-18 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,12/31/2000,23:00,12,31,2000,23,00,2000-12-31 23:00:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,12/31/2000,23:00,12,31,2000,23,00,2000-12-31 23:00:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,12/31/2000,23:45,12,31,2000,23,45,2000-12-31 23:45:00
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,12/31/2000,23:45,12,31,2000,23,45,2000-12-31 23:45:00


- We can also perform arithmetic on datetime64 dtypes with the **timedelta64** dtype


- Timedeltas are differences in datetimes, expressed in units such as days, hours, minutes, etc.


- We can convert a scalar or Series to timedelta64 with `pd.to_timedelta()`


- For example, going back to `new_df` we created earlier, we can add 1 day to each entry in `datetime` like this:

In [13]:
# Solution
(
    new_df
    .assign(
        datetime_plus_one_day=(
            lambda x: x['datetime'] + pd.to_timedelta(1, unit='D')
        )
    )
)

,City,Colors Reported,Shape Reported,State,Time,datetime,datetime_plus_one_day
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,1930-06-01 22:00:00,1930-06-02 22:00:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,1930-06-30 20:00:00,1930-07-01 20:00:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,1931-02-15 14:00:00,1931-02-16 14:00:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,1931-06-01 13:00:00,1931-06-02 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,1933-04-18 19:00:00,1933-04-19 19:00:00
...,...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,2000-12-31 23:00:00,2001-01-01 23:00:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,2000-12-31 23:00:00,2001-01-01 23:00:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,2000-12-31 23:45:00,2001-01-01 23:45:00
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,2000-12-31 23:45:00,2001-01-01 23:45:00


- [Here is the documentation for `pd.to_timedelta()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_timedelta.html)


- In particular, you can find the valid values of the `unit=...` keyword argument here

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

For the problems below, use the UFO dataset in the DataFrame `df` we created above.

### Problem 1

Add a new column to `df` containing the city for each observation, converted to all uppercase.

In [14]:
# Solution
(
    df
    .assign(
        City_Upper=lambda x: x['City'].str.upper()
    )
)

,City,Colors Reported,Shape Reported,State,Time,City_Upper
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,ITHACA
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,WILLINGBORO
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,HOLYOKE
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,ABILENE
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,NEW YORK WORLDS FAIR
...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,GRANT PARK
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,SPIRIT LAKE
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,EAGLE RIVER
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,EAGLE RIVER


### Problem 2 

Add a new column to `df` containing the city and state of each observation, separated by a comma and a space, like this: `Ithaca, NY` 

In [15]:
# Solution
(
    df
    .assign(
        City_State=lambda x: x['City'].str.cat(df['State'], sep=', ')
    )
)

,City,Colors Reported,Shape Reported,State,Time,City_State
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,"Ithaca, NY"
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,"Willingboro, NJ"
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,"Holyoke, CO"
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,"Abilene, KS"
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,"New York Worlds Fair, NY"
...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,"Grant Park, IL"
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,"Spirit Lake, IA"
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,"Eagle River, WI"
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,"Eagle River, WI"


### Problem 3

We can obtain a frequency table of the colors reported as follows:

In [16]:
(
    df
    .groupby(['Colors Reported'])
    .agg(
        count=('Colors Reported', 'count')
    )
    .sort_values('count', ascending=False)
    .reset_index()
)

,Colors Reported,count
0,RED,780
1,GREEN,531
2,ORANGE,528
3,BLUE,450
4,YELLOW,169
5,RED GREEN,89
6,RED BLUE,78
7,RED ORANGE,44
8,GREEN BLUE,34
9,RED GREEN BLUE,33


Note that the values in the column `Colors Reported` sometimes contain multiple color names. 

Drop all observations with missing values. Then filter the remaining observations for those that contain `ORANGE` in the column `Colors Reported`.

*Hint.* Remember to use `` ` ` `` to enclose a column name with spaces in the string passed to `.query()`.

In [17]:
# Solution
(
    df
    .dropna()
    .query('`Colors Reported`.str.contains("ORANGE")', engine='python')
)

,City,Colors Reported,Shape Reported,State,Time
111,Tarrant City,ORANGE,CIRCLE,AL,8/15/1949 22:00
138,Coeur d'Alene,ORANGE,CIGAR,ID,7/2/1950 13:00
174,Greenville,ORANGE,TRIANGLE,TX,4/15/1952 16:00
249,Artesia,ORANGE,OTHER,NM,8/15/1953 19:00
311,Red Bank,ORANGE,CIRCLE,NJ,12/15/1954 23:10
...,...,...,...,...,...
18144,Medina,ORANGE,TRIANGLE,OH,12/17/2000 19:30
18158,Woodland,ORANGE,LIGHT,CA,12/19/2000 23:30
18181,Fortuna,ORANGE,CIRCLE,CA,12/24/2000 18:00
18192,Atlanta,ORANGE,LIGHT,GA,12/25/2000 20:30


### Problem 4

Add a column to `df` containing the day of the week corresponding to the observation date. 

*Hint.* You may find it useful to add a column containing the observation times as datetime64 values.

In [18]:
# Solution
(
    df
    .assign(
        datetime=lambda x: pd.to_datetime(x['Time'], format='%m/%d/%Y %H:%M'),
        day_of_week=lambda x: x['datetime'].dt.dayofweek
    )
)

,City,Colors Reported,Shape Reported,State,Time,datetime,day_of_week
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,1930-06-01 22:00:00,6
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,1930-06-30 20:00:00,0
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,1931-02-15 14:00:00,6
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,1931-06-01 13:00:00,0
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,1933-04-18 19:00:00,1
...,...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,2000-12-31 23:00:00,6
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,2000-12-31 23:00:00,6
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,2000-12-31 23:45:00,6
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,2000-12-31 23:45:00,6


### Problem 5

Compute the time between observations when ordered chronologically. 

*Hint.* Convert the observation times to datetime64 values, then sort the observations based on these values. Use the `.diff()` "same size" Series method from Lesson 10 to compute the difference between consecutive rows. [Here's the documentation for `.diff()`.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html)

*Food for thought.* What's the problem with sorting the rows directly based on the values in the `Time` column?

In [19]:
# Solution
(
    df
    .assign(
        datetime=lambda x: pd.to_datetime(x['Time'], format='%m/%d/%Y %H:%M')
    )
    .sort_values('datetime', ascending=True)
    .assign(
        time_between_observations=lambda x: x['datetime'].diff(1)
    )
)

,City,Colors Reported,Shape Reported,State,Time,datetime,time_between_observations
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00,1930-06-01 22:00:00,NaT
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00,1930-06-30 20:00:00,28 days 22:00:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00,1931-02-15 14:00:00,229 days 18:00:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00,1931-06-01 13:00:00,105 days 23:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00,1933-04-18 19:00:00,687 days 06:00:00
...,...,...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00,2000-12-31 23:00:00,0 days 00:00:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00,2000-12-31 23:00:00,0 days 00:00:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45,2000-12-31 23:45:00,0 days 00:45:00
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45,2000-12-31 23:45:00,0 days 00:00:00


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Working with text data](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)
    - [Time series / date functionality](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)
    - [Time deltas](https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html)